In [2]:
# Import libraries

from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
from sklearn.preprocessing import LabelBinarizer
from six.moves import cPickle as pickle
from six.moves import range

from timeit import default_timer as timer
import librosa

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline
import seaborn as sns
current_palette = sns.color_palette()

## Feature extraction to constant shaped MFCC data

We were heavily influence on this technique used by Aqib Saeed
[here](https://github.com/aqibsaeed/Urban-Sound-Classification/blob/master/Urban%20Sound%20Classification%20using%20RNN.ipynb)
 on his blog on GitHub

import tarfile
t = tarfile.open('UrbanSound8K.tar.gz', mode="r:gz")
t.extractall()

In [13]:
raw_sound = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
raw_sound.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [12]:
fold_list = ['fold1', 'fold2', 'fold3', 'fold4', 'fold5', 'fold6', 'fold7', 'fold8', 'fold9', 'fold10']

In [9]:
from python_speech_features import logfbank
bands, frames = 20, 41
window_size = 512 * (frames - 1)
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

        


In [3]:
sound_clip, s = librosa.load('UrbanSound8K/audio/fold1/101415-3-0-2.wav')

In [10]:
# set up window size
for (start,end) in windows(sound_clip,window_size):
    if(len(sound_clip[start:end]) == window_size):
        signal = sound_clip[start:end]
        mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


In [14]:
# for each file split into 20 segments, take MFCC data from each segment.
from os import listdir, system
from os.path import isfile, join
window_size = 512 * (frames - 1)
mfcc_data = []
info = []
exception_count = 0
for i in range(10):
    mypath = 'UrbanSound8K/audio/'+ fold_list[i] + '/'
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for f in files:
        try:
            fn = mypath + f
            sound_clip, s = librosa.load(fn)
            for (start,end) in windows(sound_clip,window_size):
                if(len(sound_clip[start:end]) == window_size):
                    signal = sound_clip[start:end]
                    mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T


        except:
            print(f)
            exception_count += 1
            continue

        l_row = raw_sound.loc[raw_sound['slice_file_name']==f].values.tolist()
        label = l_row[0][-1]
        
        fold = i+1
        mfcc_data.append(mfcc)
        info.append([s, label, fold])
print("Exceptions: ", exception_count)

.DS_Store


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
Exceptions:  10


In [ ]:
# .DS_Store is a default file from Mac file systems unrelated to the Dataset

In [35]:
# can't store 3d arrays in dataframe, but this is the shape we will be useing to classify on.
#features = np.asarray(mfcc_data).reshape(len(mfcc_data),bands,frames)
#features.shape

(8732, 20, 41)

In [38]:
feat_pd = pd.DataFrame(np.array(mfcc_data).reshape(len(mfcc_data), 820))

In [39]:
feat_pd.shape

(8732, 820)

In [41]:
feat_pd['label'] = np.array(info)[:,1]
feat_pd['fold'] = np.array(info)[:,2]

In [42]:
pickle.dump(feat_pd, open('const_shape.p', 'wb'))

In [ ]:
print('done')